In [2]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **긴 문맥 재정렬(LongContextReorder)**

10개 이상의 검색된 문서를 포함할 경우 성능이 많이 떨어집니다. <br>
긴 컨텍스트 중간에 있는 관련 정보에 접근해야 할 때, 제공된 문서를 무시하는 경향이 있습니다. <br>
예를 들어서 20개의 문서 검색을 했을때 내림차순으로 배치를 하게되면 중요도를 기준으로 내림차순으로 정렬됩니다.<br>
여기서 중요도 기준으로 내림차순으로 했는데 llm이 앞부분의 문서와 끝부분의 문서를 참고한다는 것입니다. <br>
이 문제를 피하기 위해, 검색 후 문서의 순서를 재배열하여 성능이 떨어지는 것을 방지할 수 있습니다.  

`Chroma` 벡터 저장소를 사용하여 텍스트 데이터를 저장하고 검색할 수 있는 `retriever`를 생성합니다. <br>
`retriever`의 `invoke()` 메소드를 사용하여 주어진 쿼리에 대해 관련성이 높은 문서를 검색합니다.

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_transformers import LongContextReorder

In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [1]:
texts = [
    "질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다.",
    "사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.",
    "아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.",
    "챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.",
    "챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.",
    "애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.",
    "ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.",
    "비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.",
    "ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.",
    "FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.",
]

In [5]:
# 검색기를 생성합니다.
# k를 10으로 지정해서 10개가 모두 걸리도록 했습니다.
retriever = Chroma.from_texts(texts, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

검색기에 쿼리를 입력하여 검색을 수행합니다.

In [6]:
query = "ChatGPT에 대해 무엇을 말해줄 수 있나요?"

In [16]:
# 관련성이 높은 문서를 기준으로 정렬된 관련 문서를 가져옵니다.
# 질문과 관련성이 높은 문서가 위에 잡힙니다. 질문과 관련없는 문서는 하단에 잡합니다.
docs = retriever.invoke(query)

In [17]:
print(docs)

[Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'), Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'), Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'), Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'), Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'), Document(metadata={}, page_content='질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다.'), Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'), Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'), Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'), Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.')]


In [18]:
docs

[Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.'),
 Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(metadata={}, page_content='질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다.'),
 Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.')]

`LongContextReorder` 클래스의 인스턴스인 `reordering`을 생성합니다.
<br>

reordering.transform_documents(docs)를 호출하여 문서 목록 docs를 재정렬합니다.
- 덜 관련된 문서는 목록의 중간에 위치하고, 더 관련된 문서는 시작과 끝에 위치하도록 재정렬됩니다.

In [29]:
# LongContextReorder 클래스의 인스턴스인 reordering을 생성
# 검색된 문서의 내용의 순위를 재조정해주는 역할을 합니다.
reordering = LongContextReorder()

In [24]:
# transform_documents()에 docs를 넣고 문서를 재정렬합니다. 
# 관련성이 높은 문서(중요한 문서)는 시작에 위치 해있고 관련성이 떨어지는 문서는 중앙에 위치하고
# 관련성이 중간 정도의 문서는 끝에 위치. 
reordered_docs = reordering.transform_documents(docs)

In [25]:
# 4개의 관련성이 높은 문서가 시작과 끝에 위치하는지 확인합니다.
reordered_docs

[Document(metadata={}, page_content='ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.'),
 Document(metadata={}, page_content='챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.'),
 Document(metadata={}, page_content='질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다.'),
 Document(metadata={}, page_content='아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.'),
 Document(metadata={}, page_content='FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.'),
 Document(metadata={}, page_content='애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.'),
 Document(metadata={}, page_content='비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.'),
 Document(metadata={}, page_content='챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.'),
 Document(metadata={}, page_content='사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.'),
 Document(metadata={}, page_content='ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.')]

### `Context Reordering`을 사용하여 질의-응답 체인 생성

In [30]:
def format_docs(docs):
    result = []

    for i, doc in enumerate(docs):
        result.append(doc.page_content)
    
    return "\n".join(result)

In [31]:
print(format_docs(docs))

ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다.
ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다.
사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다.
챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다.
챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다.
질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다.
비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다.
아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다.
애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다.
FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다.


In [32]:
def format_docs2(docs):
    result = []

    for i, doc in enumerate(docs):
        result.append(f"[{i}] {doc.page_content} [source: yoogane_kmu@naver.com]")
    
    return "\n".join(result)

In [33]:
print(format_docs2(docs))

[0] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: yoogane_kmu@naver.com]
[1] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: yoogane_kmu@naver.com]
[2] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: yoogane_kmu@naver.com]
[3] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: yoogane_kmu@naver.com]
[4] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: yoogane_kmu@naver.com]
[5] 질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다. [source: yoogane_kmu@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: yoogane_kmu@naver.com]
[7] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: yoogane_kmu@naver.com]
[8] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: yoogane_kmu@naver.com]
[9] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: yoogane_kmu@naver.com]


In [34]:
# docs에 문서의 검색된 결과가 내림차순으로 정렬해서 들어온다. (재조정 안된 상태의 문서)
def reorder_documents(docs):            
    reordering = LongContextReorder()                       # LongContextReorder()를 만들어서     
    reordered_docs = reordering.transform_documents(docs)   # transform_documents() 에 docs를 넣어서 재조정
    combined = format_docs2(reordered_docs)                 # format_docs2() 에 순위 재조정 된 reordered_docs을 넣어서 포멧팅 되고
    print(combined)
    
    return combined                                         # 포멧팅 된 결과를 리턴

In [36]:
# 재정렬된 문서를 출력.
# 중요한 문서는 양끝(처음, 끝)에 위치
reorder_documents(docs)

[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: yoogane_kmu@naver.com]
[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: yoogane_kmu@naver.com]
[2] 질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다. [source: yoogane_kmu@naver.com]
[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: yoogane_kmu@naver.com]
[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: yoogane_kmu@naver.com]
[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: yoogane_kmu@naver.com]
[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: yoogane_kmu@naver.com]
[7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: yoogane_kmu@naver.com]
[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: yoogane_kmu@naver.com]
[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: yoogane_kmu@naver.com]


'[0] ChatGPT의 기능은 지속적인 학습과 업데이트를 통해 더욱 발전하고 있습니다. [source: yoogane_kmu@naver.com]\n[1] 챗GPT는 OpenAI에 의해 개발되었으며, 지속적으로 개선되고 있습니다. [source: yoogane_kmu@naver.com]\n[2] 질문과 관련성 있는 문장이 있을 수도 있고 없을 수도 있습니다. [source: yoogane_kmu@naver.com]\n[3] 아이폰, 아이패드, 맥북 등은 애플이 출시한 대표적인 제품들입니다. [source: yoogane_kmu@naver.com]\n[4] FIFA 월드컵은 네 번째 해마다 열리며, 국제 축구에서 가장 큰 행사입니다. [source: yoogane_kmu@naver.com]\n[5] 애플 워치와 에어팟 같은 웨어러블 기기도 애플의 인기 제품군에 속합니다. [source: yoogane_kmu@naver.com]\n[6] 비트코인은 디지털 금이라고도 불리며, 가치 저장 수단으로서 인기를 얻고 있습니다. [source: yoogane_kmu@naver.com]\n[7] 챗지피티는 사용자의 질문을 이해하고 적절한 답변을 생성하기 위해 대량의 데이터를 학습했습니다. [source: yoogane_kmu@naver.com]\n[8] 사용자와 대화하는 것처럼 설계된 AI인 ChatGPT는 다양한 질문에 답할 수 있습니다. [source: yoogane_kmu@naver.com]\n[9] ChatGPT는 복잡한 문제를 해결하거나 창의적인 아이디어를 제안하는 데에도 사용될 수 있습니다. [source: yoogane_kmu@naver.com]'